Na GCP crie uma bucket (Cloud Storage) com qualquer nome e dentro dela crie uma pasta com o nome de dados_bruto e dentro dele salve o arquivo (csv) com os dados do da base escolhida no site https://dados.gov.br/, deixe o acesso público ativado

No colab faça a leitura desse arquivo e a partir dele crie diferentes insight (pandas/pyspark/numpy) com os resultados de agrupamentos e salve esses arquivos como csv e envie diretamente pelo cada um deles para o bucket dentro de uma pasta chamada "Dados_tratados"

**Análises** - Com base nos dados obtidos pelo site gov na aba painéis de produção de etanol e biodiesel, avaliar a produção de etanol anidro e hidratado no Brasil de 2019 a 2023, os tipos de matéria-prima utilizada para a produção de etanol (obs: não há especificação do tipo de etanol produzido a partir dessa matéria-prima), capacidade de produção por estado, região e municípios, bem como a capacidade de tancagem.

OBS 2: São 4 DataFrame diferentes com dados de produção, capacidade de produção, uso de MP e tancagem.

OBS 3: Não enviei todos os df para o BigQuery, apenas os tratados com o dados de produção do etanol e de uso de MP, que pretendo usar futuramente. Os demais foram armazenados na Cloud Storage.

Bucket URL: https://console.cloud.google.com/storage/browser/analista_de_dados_soulcode

In [ ]:
!pip install --upgrade google-cloud-storage google-cloud-bigquery

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 2.8.0
    Uninstalling google-cloud-storage-2.8.0:
      Successfully uninstalled google-cloud-storage-2.8.0


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import os
from google.cloud import storage
# Inicialize o cliente
client = storage.Client()
# Defina o nome do bucket
bucket_name = 'analista_de_dados_soulcode'
bucket = client.bucket(bucket_name)
# Lista de arquivos a serem enviados
arquivos_para_enviar = ['f-etanol-producao.csv', 'h-etanol-materia-prima.csv', 'e-etanol-capacidade.csv', 'g-etanol-tancagem.csv' ]  # Adicione os nomes dos seus arquivos aqui

# Caminho local onde os arquivos estão armazenados
caminho_local_dos_arquivos = '/content/drive/MyDrive/Colab Notebooks/Datasets'  # Substitua pelo caminho real

for nome_arquivo in arquivos_para_enviar:
    caminho_completo_do_arquivo = os.path.join(caminho_local_dos_arquivos, nome_arquivo)
    destination_blob_path = f'dados_brutos/{nome_arquivo}'
    blob = bucket.blob(destination_blob_path)
    blob.upload_from_filename(caminho_completo_do_arquivo)
    print(f'Arquivo {nome_arquivo} enviado para o Cloud Storage na pasta csv com sucesso!')

Arquivo f-etanol-producao.csv enviado para o Cloud Storage na pasta csv com sucesso!
Arquivo h-etanol-materia-prima.csv enviado para o Cloud Storage na pasta csv com sucesso!
Arquivo e-etanol-capacidade.csv enviado para o Cloud Storage na pasta csv com sucesso!
Arquivo g-etanol-tancagem.csv enviado para o Cloud Storage na pasta csv com sucesso!


In [ ]:
from google.cloud import storage
import pandas as pd
import io

bucket_name = 'analista_de_dados_soulcode'
file_name = 'dados_brutos/f-etanol-producao.csv'
# Obtenha o bucket e o blob (arquivo)
bucket = client.bucket(bucket_name)
blob = bucket.blob(file_name)
# Faça o download do arquivo como bytes
data = blob.download_as_bytes()

# Carregue os dados em um DataFrame do pandas
df_producao = pd.read_csv(io.BytesIO(data))

# Limpeza dos dados
df_producao = df_producao.drop_duplicates()
print(df_producao.isnull().sum())
df_producao.columns = df_producao.columns.str.title()
df_producao = df_producao.dropna()

df_producao.head()

Mês/Ano                              0
Região                               0
Estado                               0
Produção Etanol Hidratado(m³/d)      6
Produção Etanol Anidro (m³/d)      142
dtype: int64


,Mês/Ano,Região,Estado,Produção Etanol Hidratado(M³/D),Produção Etanol Anidro (M³/D)
0,08/2023,CENTRO OESTE,Goiás,661956.0,167493.0
1,08/2023,CENTRO OESTE,Mato Grosso,346609.0,227672.0
2,08/2023,CENTRO OESTE,Mato Grosso do Sul,378912.0,119057.0
4,08/2023,NORDESTE,Bahia,32894.0,22560.0
5,08/2023,NORDESTE,Maranhão,6067.0,28060.0


In [ ]:
#Arquivo tratado para subir no BigQuery

df_producao_tratado = df_producao.rename(columns={
    'Mês/Ano': 'Mês_Ano',
    'Produção Etanol Hidratado(M³/D)': 'Produção Etanol Hidratado',
    'Produção Etanol Anidro (M³/D)': 'Produção Etanol Anidro'
})
df_producao_tratado['Mês_Ano'] = pd.to_datetime(df_producao_tratado['Mês_Ano'], format='%m/%Y')
df_producao_tratado['Ano'] = df_producao_tratado['Mês_Ano'].dt.year
df_producao_tratado['Mês'] = df_producao_tratado['Mês_Ano'].dt.month

df_producao_tratado.head()

#Envio para o cloud
df_producao_tratado.to_csv('producao_etanol_tratado.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/producao_etanol_tratado.csv')
processed_blob.upload_from_filename('producao_etanol_tratado.csv')

#Enviar do DataFrame para o BigQuery
# Definição do ID do projeto, conjunto de dados e tabela
project_id = 'xenon-region-434413-s1'
dataset_id = 'dados_soulcode_curso'
table_id = 'dados-producao-etanol'

# Envio do DataFrame para o BigQuery
table_ref = f'{project_id}.{dataset_id}.{table_id}'
df_producao_tratado.to_gbq(destination_table=table_ref, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 6647.07it/s]


In [ ]:
#df com dados de matéria-prima
file_name2 = 'dados_brutos/h-etanol-materia-prima.csv'
bucket = client.bucket(bucket_name)
blob = bucket.blob(file_name2)
# Faça o download do arquivo como bytes
data = blob.download_as_bytes()

# Carregue os dados em um DataFrame do pandas
df_mp = pd.read_csv(io.BytesIO(data))

# Limpeza
df_mp = df_mp.drop_duplicates()
df_mp.isnull().sum()
df_mp.columns = df_mp.columns.str.title()
df_mp = df_mp.dropna()
df_mp.head()


,Mês/Ano,Região,Estado,Produto,Quantidade Processada (T)
0,08/2023,SUDESTE,São Paulo,BAGAÇO OU PALHA DE CANA,12713
1,08/2023,CENTRO OESTE,Goiás,CANA DE AÇÚCAR,7422467
2,08/2023,CENTRO OESTE,Mato Grosso,CANA DE AÇÚCAR,1544723
3,08/2023,CENTRO OESTE,Mato Grosso do Sul,CANA DE AÇÚCAR,4378373
4,08/2023,NORDESTE,Alagoas,CANA DE AÇÚCAR,4857


In [ ]:
#Arquivo tratado para subir no BigQuery

df_mp_tratado = df_mp.rename(columns={
    'Mês/Ano': 'Mês_Ano',
    'Quantidade Processada (T)' : 'Quantidade Processada'
})
df_mp_tratado['Mês_Ano'] = pd.to_datetime(df_mp_tratado['Mês_Ano'], format='%m/%Y')
df_mp_tratado['Ano'] = df_mp_tratado['Mês_Ano'].dt.year
df_mp_tratado['Mês'] = df_mp_tratado['Mês_Ano'].dt.month

df_mp_tratado.head()
df_mp_tratado.to_csv('materia_prima_etanol_tratado.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/materia_prima_etanol_tratado.csv')
processed_blob.upload_from_filename('materia_prima_etanol_tratado.csv')

#Enviar do DataFrame para o BigQuery
# Definição do ID do projeto, conjunto de dados e tabela
project_id = 'xenon-region-434413-s1'
dataset_id = 'dados_soulcode_curso'
table_id = 'dados-mp-etanol'

# Envio do DataFrame para o BigQuery
table_ref = f'{project_id}.{dataset_id}.{table_id}'
df_mp_tratado.to_gbq(destination_table=table_ref, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 717.83it/s]


In [ ]:
#df com dados de capacidade
file_name3 = 'dados_brutos/e-etanol-capacidade.csv'
bucket = client.bucket(bucket_name)
blob = bucket.blob(file_name3)
# Faça o download do arquivo como bytes
data = blob.download_as_bytes()

# Carregue os dados em um DataFrame do pandas
df_capacidade = pd.read_csv(io.BytesIO(data))

# Limpeza
df_capacidade = df_capacidade.drop_duplicates()
df_capacidade.isnull().sum()
df_capacidade.columns = df_capacidade.columns.str.title()
df_capacidade = df_capacidade.dropna()
df_capacidade.head()

,Mês/Ano,Razão Social,Cnpj,Região,Estado,Município,Capacidade Produção Etanol Anidro (M³/D),Capacidade Produção Etanol Hidratado (M³/D)
0,03/2018,ABENGOA BIOENERGIA AGROINDUSTRIA LTDA - EM REC...,6252818003446,SUDESTE,São Paulo,PIRASSUNUNGA,200,600
1,04/2018,ABENGOA BIOENERGIA AGROINDUSTRIA LTDA - EM REC...,6252818003446,SUDESTE,São Paulo,PIRASSUNUNGA,200,600
2,05/2018,ABENGOA BIOENERGIA AGROINDUSTRIA LTDA - EM REC...,6252818003446,SUDESTE,São Paulo,PIRASSUNUNGA,200,600
3,06/2018,ABENGOA BIOENERGIA AGROINDUSTRIA LTDA - EM REC...,6252818003446,SUDESTE,São Paulo,PIRASSUNUNGA,200,600
4,07/2018,ABENGOA BIOENERGIA AGROINDUSTRIA LTDA - EM REC...,6252818003446,SUDESTE,São Paulo,PIRASSUNUNGA,200,600


In [ ]:
#Arquivo tratado para subir no BigQuery

df_capacidade_tratado = df_capacidade.rename(columns={
    'Mês/Ano': 'Mês_Ano',
    'Capacidade Produção Etanol Anidro (M³/D)': 'Capacidade Produção Etanol Anidro',
    'Capacidade Produção Etanol Hidratado (M³/D)': 'Capacidade Produção Etanol Hidratado'
})
df_capacidade_tratado['Mês_Ano'] = pd.to_datetime(df_capacidade_tratado['Mês_Ano'], format='%m/%Y')
df_capacidade_tratado['Ano'] = df_capacidade_tratado['Mês_Ano'].dt.year
df_capacidade_tratado['Mês'] = df_capacidade_tratado['Mês_Ano'].dt.month

df_capacidade_tratado.head()
df_capacidade_tratado.to_csv('capacidade_etanol_tratado.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/capacidade_etanol_tratado.csv')
processed_blob.upload_from_filename('capacidade_etanol_tratado.csv')

In [ ]:
#df com dados de tancagem
file_name4 = 'dados_brutos/g-etanol-tancagem.csv'
bucket = client.bucket(bucket_name)
blob = bucket.blob(file_name4)
# Faça o download do arquivo como bytes
data = blob.download_as_bytes()

# Carregue os dados em um DataFrame do pandas
df_tancagem = pd.read_csv(io.BytesIO(data))

# Limpeza
df_tancagem = df_tancagem.drop_duplicates()
df_tancagem.isnull().sum()
df_tancagem.columns = df_tancagem.columns.str.title()
#df_tancagem = df_tancagem.dropna()
df_tancagem.head()

,Mês/Ano,Razão Social,Cnpj,Região,Estado,Município,Tancagem Etanol Hidratado (M³),Tancagem Etanol Anidro (M³),Tancagem Outros Alcóois (M³)
0,08/2023,CARAMURU ALIMENTOS S/A,80671002668,CENTRO OESTE,Mato Grosso,SORRISO,6000.0,NaN,NaN
1,08/2023,DESTILARIA DE ALCOOL LIBRA LTDA,297598000122,CENTRO OESTE,Mato Grosso,SAO JOSE DO RIO CLARO,47000.0,NaN,NaN
2,08/2023,CENTRAL ENERGETICA VALE DO SAPUCAI LTDA,372496000124,SUDESTE,São Paulo,PATROCINIO PAULISTA,50000.0,NaN,NaN
3,08/2023,DENUSA DESTILARIA NOVA UNIAO S/A - EM RECUPERA...,595322000120,CENTRO OESTE,Goiás,JANDAIA,60000.0,NaN,NaN
4,08/2023,SANTA CRUZ ACUCAR E ALCOOL LTDA,738822000255,NORDESTE,Bahia,SANTA CRUZ CABRALIA,10000.0,5000.0,NaN


In [ ]:
#Arquivo tratado para subir no BigQuery

df_tancagem_tratado = df_tancagem.rename(columns={
    'Mês/Ano': 'Mês_Ano',
    'Tancagem Etanol Hidratado (M³)': 'Tancagem Etanol Hidratado',
    'Tancagem Etanol Anidro (M³)': 'Tancagem Etanol Anidro',
    'Tancagem Outros Alcóois (M³)' : 'Tancagem Outros Alcóois'
})
df_tancagem_tratado['Mês_Ano'] = pd.to_datetime(df_tancagem_tratado['Mês_Ano'], format='%m/%Y')
df_tancagem_tratado['Ano'] = df_tancagem_tratado['Mês_Ano'].dt.year
df_tancagem_tratado['Mês'] = df_tancagem_tratado['Mês_Ano'].dt.month

df_tancagem_tratado.head()
df_tancagem_tratado.to_csv('tancagem_etanol_tratado.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/tancagem_etanol_tratado.csv')
processed_blob.upload_from_filename('tancagem_etanol_tratado.csv')

**1** - Produção de etanol hidratado e anidro no Brasil por mês de 2019 a 2023

In [ ]:
# Etanol Anidro

df_producao['Mês/Ano'] = pd.to_datetime(df_producao['Mês/Ano'], format='%m/%Y')
df_producao['Ano'] = df_producao['Mês/Ano'].dt.year
df_producao['Mês'] = df_producao['Mês/Ano'].dt.month

producao_mensal_anidro = df_producao.groupby(['Ano', 'Mês'])['Produção Etanol Anidro (M³/D)'].sum().reset_index(name = "Produção Etanol Anidro (M³/D)")
producao_mensal_anidro = round(producao_mensal_anidro, 2)
print(producao_mensal_anidro)
producao_mensal_anidro.to_csv('producao_mensal_anidro.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/producao_mensal_anidro.csv')
processed_blob.upload_from_filename('producao_mensal_anidro.csv')

     Ano  Mês  Produção Etanol Anidro (M³/D)
0   2019    1                       134879.0
1   2019    2                        67328.0
2   2019    3                       104452.0
3   2019    4                       422938.0
4   2019    5                      1247647.0
5   2019    6                      1504380.0
6   2019    7                      1552913.0
7   2019    8                      1568563.0
8   2019    9                      1337736.0
9   2019   10                      1412341.0
10  2019   11                       805905.0
11  2019   12                       235172.0
12  2020    1                       151268.0
13  2020    2                        75987.0
14  2020    3                       142132.0
15  2020    4                       615541.0
16  2020    5                      1060050.0
17  2020    6                      1184708.0
18  2020    7                      1469321.0
19  2020    8                      1450988.0
20  2020    9                      1617403.0
21  2020  

In [ ]:
# Etanol hidratado

producao_mensal_hidratado = df_producao.groupby(['Ano', 'Mês'])['Produção Etanol Hidratado(M³/D)'].sum().reset_index(name = "Produção Etanol Hidratado (M³/D)")
resultado = round(producao_mensal_hidratado, 2)
print(resultado)
producao_mensal_hidratado.to_csv('producao_mensal_hidratado.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/producao_mensal_hidratado.csv')
processed_blob.upload_from_filename('producao_mensal_hidratado.csv')

     Ano  Mês  Produção Etanol Hidratado (M³/D)
0   2019    1                          218427.0
1   2019    2                          158681.0
2   2019    3                          471761.0
3   2019    4                         1849021.0
4   2019    5                         2796146.0
5   2019    6                         3064382.0
6   2019    7                         3404262.0
7   2019    8                         3784037.0
8   2019    9                         3548800.0
9   2019   10                         3341073.0
10  2019   11                         1660643.0
11  2019   12                          492701.0
12  2020    1                          323719.0
13  2020    2                          236518.0
14  2020    3                          522999.0
15  2020    4                         1964575.0
16  2020    5                         2651434.0
17  2020    6                         2711582.0
18  2020    7                         3170644.0
19  2020    8                         31

**2** - Produção total e média de etanol hidratado e anidro por região do Brasil

In [ ]:
# Etanol hidratado produção total

producao_total_hidratado = df_producao.groupby('Região')['Produção Etanol Hidratado(M³/D)'].sum().reset_index(name = "Produção Etanol Hidratado (M³/D)")
producao_total_hidratado = round(producao_total_hidratado, 2)
producao_total_hidratado = producao_total_hidratado.sort_values(by='Produção Etanol Hidratado (M³/D)', ascending = False)
print(producao_total_hidratado)
producao_total_hidratado.to_csv('producao_total_hidratado.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/producao_total_hidratado.csv')
processed_blob.upload_from_filename('producao_total_hidratado.csv')

         Região  Produção Etanol Hidratado (M³/D)
3       SUDESTE                        47648732.0
0  CENTRO OESTE                        40778931.0
1      NORDESTE                         4432298.0
4           SUL                         3248073.0
2         NORTE                          475147.0


In [ ]:
# Etanol hidratado produção média

producao_media_hidratado = df_producao.groupby('Região')['Produção Etanol Hidratado(M³/D)'].mean().reset_index(name = "Produção Média De Etanol Hidratado (M³/D)")
producao_media_hidratado = round(producao_media_hidratado, 2)
producao_media_hidratado = producao_media_hidratado.sort_values(by='Produção Média De Etanol Hidratado (M³/D)', ascending = False)
print(producao_media_hidratado)
producao_media_hidratado.to_csv('producao_media_hidratado.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/producao_media_hidratado.csv')
processed_blob.upload_from_filename('producao_media_hidratado.csv')

         Região  Produção Média De Etanol Hidratado (M³/D)
3       SUDESTE                                  333207.92
0  CENTRO OESTE                                  248652.02
4           SUL                                   63687.71
1      NORDESTE                                   16001.08
2         NORTE                                    7091.75


In [ ]:
# Etanol anidro produção total

producao_total_anidro = df_producao.groupby('Região')['Produção Etanol Anidro (M³/D)'].sum().reset_index(name = "Produção Etanol Anidro (M³/D)")
producao_total_anidro = round(producao_total_anidro, 2)
producao_total_anidro = producao_total_anidro.sort_values(by='Produção Etanol Anidro (M³/D)', ascending = False)
print(producao_total_anidro)
producao_total_anidro.to_csv('producao_total_anidro.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/producao_total_anidro.csv')
processed_blob.upload_from_filename('producao_total_anidro.csv')

         Região  Produção Etanol Anidro (M³/D)
3       SUDESTE                     31094888.0
0  CENTRO OESTE                     14847046.0
1      NORDESTE                      3975862.0
4           SUL                      2759378.0
2         NORTE                       611194.0


In [ ]:
# Etanol anidro produção média

producao_media_anidro = df_producao.groupby('Região')['Produção Etanol Anidro (M³/D)'].mean().reset_index(name = "Produção Média De Etanol Anidro(M³/D)")
producao_media_anidro = round(producao_media_anidro, 2)
producao_media_anidro = producao_media_anidro.sort_values(by='Produção Média De Etanol Anidro(M³/D)', ascending = False)
print(producao_media_anidro)
producao_media_anidro.to_csv('producao_media_anidro.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/producao_media_anidro.csv')
processed_blob.upload_from_filename('producao_media_anidro.csv')

         Região  Produção Média De Etanol Anidro(M³/D)
3       SUDESTE                              217446.77
0  CENTRO OESTE                               90530.77
4           SUL                               54105.45
1      NORDESTE                               14353.29
2         NORTE                                9122.30


**3** - Estado com maior produção de etanol anidro e hidratado

In [ ]:
#Identificando a produção de cada estado
producao_estado_anidro= df_producao.groupby('Estado')['Produção Etanol Anidro (M³/D)'].sum().reset_index(name = "Produção De Etanol Anidro(M³/D)")
producao_estado_anidro = round(producao_estado_anidro, 2)
producao_estado_anidro = producao_estado_anidro.sort_values(by='Produção De Etanol Anidro(M³/D)', ascending = False)
print(producao_estado_anidro)
producao_estado_anidro.to_csv('producao_estado_anidro.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/producao_estado_anidro.csv')
processed_blob.upload_from_filename('producao_estado_anidro.csv')

#Estado com maior e menor produção de etanol anidro

estado_maior_producao = producao_estado_anidro.index[0]
maior_producao = producao_estado_anidro.iloc[0]
print(f"O estado com a maior produção de etanol anidro é {estado_maior_producao} com uma produção total de {maior_producao:} m³.")

producao_estado_anidro_cresc = producao_estado_anidro.sort_values(by='Produção De Etanol Anidro(M³/D)', ascending = True)
estado_menor_producao = producao_estado_anidro_cresc.index[0]
menor_producao = producao_estado_anidro_cresc.iloc[0]
print(f"O estado com a menor produção de etanol anidro é {estado_menor_producao} com uma produção total de {menor_producao:} m³.")


                 Estado  Produção De Etanol Anidro(M³/D)
15            São Paulo                       25435649.0
5           Mato Grosso                        5752132.0
7          Minas Gerais                        5256391.0
3                 Goiás                        5046165.0
6    Mato Grosso do Sul                        4048749.0
8                Paraná                        2759378.0
9               Paraíba                         968830.0
0               Alagoas                         894576.0
4              Maranhão                         665086.0
11           Pernambuco                         567730.0
1                 Bahia                         546334.0
16            Tocantins                         426072.0
2        Espírito Santo                         402848.0
10                 Pará                         185122.0
13  Rio Grande do Norte                         135056.0
12                Piauí                         111820.0
14              Sergipe        

In [ ]:
#Identificando a produção de cada estado
producao_estado_hidratado= df_producao.groupby('Estado')['Produção Etanol Hidratado(M³/D)'].sum().reset_index(name = "Produção De Etanol Hidratado(M³/D)")
producao_estado_hidratado = round(producao_estado_hidratado, 2)
producao_estado_hidratado = producao_estado_hidratado.sort_values(by='Produção De Etanol Hidratado(M³/D)', ascending = False)
print(producao_estado_hidratado)
producao_estado_hidratado.to_csv('producao_estado_hidratado.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/producao_estado_hidratado.csv')
processed_blob.upload_from_filename('producao_estado_hidratado.csv')

#Estado com maior e menor produção de etanol hidratado

estado_maior_producao = producao_estado_hidratado.index[0]
maior_producao = producao_estado_hidratado.iloc[0]
print(f"O estado com a maior produção de etanol hidratado é {estado_maior_producao} com uma produção total de {maior_producao:} m³.")

producao_estado_hidratado_cresc = producao_estado_hidratado.sort_values(by='Produção De Etanol Hidratado(M³/D)', ascending = True)
estado_menor_producao = producao_estado_hidratado_cresc.index[0]
menor_producao = producao_estado_hidratado_cresc.iloc[0]
print(f"O estado com a menor produção de etanol hidratado é {estado_menor_producao} com uma produção total de {menor_producao:} m³.")

                 Estado  Produção De Etanol Hidratado(M³/D)
15            São Paulo                          38381393.0
3                 Goiás                          19149443.0
5           Mato Grosso                          11314370.0
6    Mato Grosso do Sul                          10315118.0
7          Minas Gerais                           9206469.0
8                Paraná                           3248073.0
0               Alagoas                           1096037.0
11           Pernambuco                            973878.0
1                 Bahia                            848054.0
9               Paraíba                            685686.0
16            Tocantins                            412777.0
14              Sergipe                            342959.0
13  Rio Grande do Norte                            333330.0
4              Maranhão                             76919.0
12                Piauí                             75435.0
10                 Pará                 

**4** -  Época com maior produção de cada tipo de etanol

In [ ]:
#Etanol anidro

producao_epoca_anidro= df_producao.groupby('Ano')['Produção Etanol Anidro (M³/D)'].max().reset_index(name = "Produção De Etanol Anidro(M³/D)")
producao_epoca_anidro = round(producao_epoca_anidro, 2)
producao_epoca_anidro = producao_epoca_anidro.sort_values(by='Produção De Etanol Anidro(M³/D)', ascending = False)
print(producao_epoca_anidro)
producao_epoca_anidro.to_csv('producao_epoca_anidro.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/producao_epoca_anidro.csv')
processed_blob.upload_from_filename('producao_epoca_anidro.csv')

    Ano  Produção De Etanol Anidro(M³/D)
2  2021                        1008391.0
3  2022                         970906.0
4  2023                         953919.0
0  2019                         904786.0
1  2020                         815876.0


In [ ]:
# Etanol hidratado

producao_epoca_hidratado= df_producao.groupby('Ano')['Produção Etanol Hidratado(M³/D)'].max().reset_index(name = "Produção De Etanol Hidratado(M³/D)")
producao_epoca_hidratado = round(producao_epoca_hidratado, 2)
producao_epoca_hidratado = producao_epoca_hidratado.sort_values(by='Produção De Etanol Hidratado(M³/D)', ascending = False)
print(producao_epoca_hidratado)
producao_epoca_hidratado.to_csv('producao_epoca_hidratado.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/producao_epoca_hidratado.csv')
processed_blob.upload_from_filename('producao_epoca_hidratado.csv')

    Ano  Produção De Etanol Hidratado(M³/D)
0  2019                           1758127.0
1  2020                           1475375.0
2  2021                           1135592.0
3  2022                           1132409.0
4  2023                           1105928.0


**5** - Avaliar o uso de cada matéria-prima por ano.

In [ ]:
df_mp['Mês/Ano'] = pd.to_datetime(df_mp['Mês/Ano'], format='%m/%Y')
df_mp['Ano'] = df_mp['Mês/Ano'].dt.year
df_mp['Mês'] = df_mp['Mês/Ano'].dt.month
#print(df_mp)

materia_prima_ano = df_mp.groupby(['Produto', 'Ano'])['Quantidade Processada (T)'].sum().reset_index(name = "Quantidade de Matéria-Prima")
materia_prima_ano= round(materia_prima_ano, 2)
materia_prima_ano = materia_prima_ano.sort_values(by='Produto', ascending = True)
print(materia_prima_ano)
materia_prima_ano.to_csv('materia_prima_ano.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/materia_prima_ano.csv')
processed_blob.upload_from_filename('materia_prima_ano.csv')

                     Produto   Ano  Quantidade de Matéria-Prima
0    BAGAÇO OU PALHA DE CANA  2019                       105340
1    BAGAÇO OU PALHA DE CANA  2020                       138184
2    BAGAÇO OU PALHA DE CANA  2021                       101130
3    BAGAÇO OU PALHA DE CANA  2022                        49130
4    BAGAÇO OU PALHA DE CANA  2023                        12713
5             CANA DE AÇÚCAR  2019                    379586710
6             CANA DE AÇÚCAR  2020                    325792775
7             CANA DE AÇÚCAR  2021                    284594461
8             CANA DE AÇÚCAR  2022                    291418890
9             CANA DE AÇÚCAR  2023                    293362346
14                    MELAÇO  2023                      7925386
13                    MELAÇO  2022                     10763187
12                    MELAÇO  2021                     11954703
11                    MELAÇO  2020                     10707410
10                    MELAÇO  2019      

**6** - Matéria-prima mais utilizada por região.

In [ ]:
materia_prima_regiao = df_mp.groupby(['Região', 'Produto'])['Quantidade Processada (T)'].sum().reset_index(name = "Quantidade de Matéria-Prima")
materia_prima_regiao= round(materia_prima_regiao, 2)
materia_prima_regiao = materia_prima_regiao.sort_values(by='Quantidade de Matéria-Prima', ascending = False)
print(materia_prima_regiao)
materia_prima_regiao.to_csv('materia_prima_regiao.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/materia_prima_regiao.csv')
processed_blob.upload_from_filename('materia_prima_regiao.csv')

          Região                   Produto  Quantidade de Matéria-Prima
11       SUDESTE            CANA DE AÇÚCAR                    859055598
0   CENTRO OESTE            CANA DE AÇÚCAR                    459001479
5       NORDESTE            CANA DE AÇÚCAR                    174120841
15           SUL            CANA DE AÇÚCAR                     68080454
12       SUDESTE                    MELAÇO                     36733901
2   CENTRO OESTE                     MILHO                     34310243
8          NORTE            CANA DE AÇÚCAR                     14496810
6       NORDESTE                    MELAÇO                      6640304
1   CENTRO OESTE                    MELAÇO                      3078195
18           SUL  OUTRAS MATÉRIAS - PRIMAS                      1807479
14       SUDESTE  OUTRAS MATÉRIAS - PRIMAS                      1255610
16           SUL                    MELAÇO                       522347
10       SUDESTE   BAGAÇO OU PALHA DE CANA                      

**7** - Média de cana-de-açúcar processada por ano na região sudeste.

In [ ]:
df_mp_filtrado = df_mp[(df_mp['Produto'] == 'CANA DE AÇÚCAR') & (df_mp['Região'] == 'SUDESTE')]
df_mp_filtrado.head()

df_cana_media = df_mp_filtrado.groupby(['Ano', 'Estado'])['Quantidade Processada (T)'].mean().reset_index(name = "Quantidade de Matéria-Prima")
df_cana_media= round(df_cana_media, 2)
df_cana_media = df_cana_media.sort_values(by='Quantidade de Matéria-Prima', ascending = False)
print(df_cana_media)
df_cana_media.to_csv('cana_quantidade_media.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/cana_quantidade_media.csv')
processed_blob.upload_from_filename('cana_quantidade_media.csv')

     Ano          Estado  Quantidade de Matéria-Prima
3   2019       São Paulo                  15725048.58
19  2023       São Paulo                  13781706.43
15  2022       São Paulo                  13580404.00
7   2020       São Paulo                  13184687.83
11  2021       São Paulo                  10837003.25
1   2019    Minas Gerais                   3248595.91
5   2020    Minas Gerais                   2700118.30
13  2022    Minas Gerais                   2659426.40
17  2023    Minas Gerais                   2605183.50
9   2021    Minas Gerais                   2555202.00
6   2020  Rio de Janeiro                    203920.33
12  2022  Espírito Santo                    199529.12
18  2023  Rio de Janeiro                    196139.25
14  2022  Rio de Janeiro                    194403.71
10  2021  Rio de Janeiro                    191745.56
2   2019  Rio de Janeiro                    188046.14
16  2023  Espírito Santo                    179544.60
8   2021  Espírito Santo    

**8** - Empresas com maior capacidade de produção de etanol anidro e hidratado

In [ ]:
producao_anidro = df_capacidade.groupby('Razão Social')['Capacidade Produção Etanol Anidro (M³/D)'].max().reset_index(name = "Capacidade de Produção")
producao_anidro = producao_anidro.sort_values(by='Capacidade de Produção', ascending = False)
print(producao_anidro)
producao_anidro.to_csv('producao_anidro.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/producao_anidro.csv')
processed_blob.upload_from_filename('producao_anidro.csv')

                                          Razão Social  Capacidade de Produção
148                          INPASA AGROINDUSTRIAL S/A                    3000
135              FS INDÚSTRIA DE BIOCOMBUSTÍVEIS LTDA.                    2500
191                                 RAIZEN ENERGIA S.A                    1710
134                      FS I INDÚSTRIA DE ETANOL S.A.                    1700
211                                   SAO MARTINHO S/A                    1700
..                                                 ...                     ...
139                          GUARIROBA BIOENERGIA LTDA                       0
142  IMCOPA - IMPORTACAO, EXPORTACAO E INDUSTRIA DE...                       0
144   INDUSTRIA E COMERCIO DE BEBIDAS SEIS LAGOAS LTDA                       0
145  INDUSTRIA E COMERCIO DESTILARIA MANTO AZUL EIRELI                       0
311  ÁLCOOL QUÍMICA CANABRAVA S.A. EM RECUPERAÇÃO J...                       0

[312 rows x 2 columns]


In [ ]:
producao_hidratado = df_capacidade.groupby('Razão Social')['Capacidade Produção Etanol Hidratado (M³/D)'].max().reset_index(name = "Capacidade de Produção")
producao_hidratado = producao_hidratado.sort_values(by='Capacidade de Produção', ascending = False)
print(producao_hidratado)
producao_anidro.to_csv('producao_hidratado.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/producao_hidratado.csv')
processed_blob.upload_from_filename('producao_hidratado.csv')

                                          Razão Social  Capacidade de Produção
211                                   SAO MARTINHO S/A                    3120
148                          INPASA AGROINDUSTRIAL S/A                    3000
61                          CERRADINHO BIOENERGIA S.A.                    2800
7    AGRO ENERGIA SANTA LUZIA S.A. - EM RECUPERACAO...                    2700
135              FS INDÚSTRIA DE BIOCOMBUSTÍVEIS LTDA.                    2600
..                                                 ...                     ...
48                               CANEX BIOENERGIA LTDA                       1
174  NEXUS PRODUCAO E COMERCIALIZACAO DE BIOENERGIA...                       1
63         CHS AGRONEGOCIO - INDUSTRIA E COMERCIO LTDA                       1
223           TRIO COMERCIO DE ACUCAR E LOGISTICA LTDA                       1
64                   CIA AGROPASTORIL VALE DA PIRAGIBA                       0

[312 rows x 2 columns]


**9** - Capacidade de produção média por região.

In [ ]:
df_capacidade['Mês/Ano'] = pd.to_datetime(df_capacidade['Mês/Ano'], format='%m/%Y')
df_capacidade['Ano'] = df_capacidade['Mês/Ano'].dt.year
df_capacidade['Mês'] = df_capacidade['Mês/Ano'].dt.month


producao_anidro_media = df_capacidade.groupby(['Região', 'Ano'])['Capacidade Produção Etanol Anidro (M³/D)'].mean().reset_index(name = "Capacidade de Produção")
producao_anidro_media= round(producao_anidro_media, 2)
producao_anidro_media = producao_anidro_media.sort_values(by='Capacidade de Produção', ascending = False)
print(producao_anidro_media)
producao_anidro.to_csv('producao_anidro_media.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/producao_anidro_media.csv')
processed_blob.upload_from_filename('producao_anidro_media.csv')

          Região   Ano  Capacidade de Produção
5   CENTRO OESTE  2023                  542.68
4   CENTRO OESTE  2022                  532.66
3   CENTRO OESTE  2021                  488.94
2   CENTRO OESTE  2020                  471.44
1   CENTRO OESTE  2019                  444.66
0   CENTRO OESTE  2018                  416.21
23       SUDESTE  2023                  393.08
22       SUDESTE  2022                  389.97
20       SUDESTE  2020                  389.33
21       SUDESTE  2021                  388.22
19       SUDESTE  2019                  387.27
18       SUDESTE  2018                  384.29
12         NORTE  2018                  262.64
17         NORTE  2023                  262.00
16         NORTE  2022                  262.00
15         NORTE  2021                  262.00
14         NORTE  2020                  262.00
13         NORTE  2019                  262.00
27           SUL  2021                  255.81
28           SUL  2022                  255.81
26           

In [ ]:
producao_hidratado_media = df_capacidade.groupby(['Região', 'Ano'])['Capacidade Produção Etanol Hidratado (M³/D)'].mean().reset_index(name = "Capacidade de Produção")
producao_hidratado_media= round(producao_hidratado_media, 2)
producao_hidratado_media = producao_hidratado_media.sort_values(by='Capacidade de Produção', ascending = False)
print(producao_hidratado_media)
producao_anidro.to_csv('producao_hidratado_media.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/producao_hidratado_media.csv')
processed_blob.upload_from_filename('producao_hidratado_media.csv')

          Região   Ano  Capacidade de Produção
4   CENTRO OESTE  2022                 1052.02
5   CENTRO OESTE  2023                 1050.22
3   CENTRO OESTE  2021                  996.59
2   CENTRO OESTE  2020                  966.33
1   CENTRO OESTE  2019                  935.83
0   CENTRO OESTE  2018                  899.88
23       SUDESTE  2023                  716.22
22       SUDESTE  2022                  704.83
21       SUDESTE  2021                  699.18
20       SUDESTE  2020                  695.06
19       SUDESTE  2019                  680.90
18       SUDESTE  2018                  673.86
17         NORTE  2023                  506.00
16         NORTE  2022                  506.00
12         NORTE  2018                  503.02
15         NORTE  2021                  483.50
14         NORTE  2020                  476.00
13         NORTE  2019                  476.00
26           SUL  2020                  474.85
27           SUL  2021                  471.16
28           

**10** - Avaliar se houve aumento ou redução da capacidade de produção ao longo dos anos

In [ ]:
producao_anidro_anos = df_capacidade.groupby('Ano')['Capacidade Produção Etanol Anidro (M³/D)'].sum().reset_index(name = "Capacidade de Produção")
producao_anidro_anos= round(producao_anidro_anos, 2)
producao_anidro_anos = producao_anidro_anos.sort_values(by='Ano', ascending = True)
print(producao_anidro_anos)
producao_anidro.to_csv('producao_anidro_anos.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/producao_anidro_anos.csv')
processed_blob.upload_from_filename('producao_anidro_anos.csv')

    Ano  Capacidade de Produção
0  2018                 1431989
1  2019                 1548956
2  2020                 1578370
3  2021                 1589950
4  2022                 1630304
5  2023                 1103476


In [ ]:
producao_hidratado_anos = df_capacidade.groupby('Ano')['Capacidade Produção Etanol Hidratado (M³/D)'].sum().reset_index(name = "Capacidade de Produção")
producao_hidratado_anos= round(producao_hidratado_anos, 2)
producao_hidratado_anos = producao_hidratado_anos.sort_values(by='Ano', ascending = True)
print(producao_hidratado_anos)
producao_anidro.to_csv('producao_hidratado_anos.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/producao_hidratado_anos.csv')
processed_blob.upload_from_filename('producao_hidratado_anos.csv')

    Ano  Capacidade de Produção
0  2018                 2622377
1  2019                 2833387
2  2020                 2902980
3  2021                 2939275
4  2022                 3000250
5  2023                 2030048


**11** - Ordenar as empresas com  maior capacidade tancagem de cada etanol

In [ ]:
df_nulos_removidos = df_tancagem.dropna(subset=['Tancagem Etanol Anidro (M³)'])

tancagem_anidro_empresa = df_nulos_removidos.groupby('Razão Social')['Tancagem Etanol Anidro (M³)'].max().reset_index(name = "Capacidade de Tancagem")
tancagem_anidro_empresa = round(tancagem_anidro_empresa, 2)
tancagem_anidro_empresa = tancagem_anidro_empresa.sort_values(by='Capacidade de Tancagem', ascending = False)
print(tancagem_anidro_empresa)
tancagem_anidro_empresa.to_csv('tancagem_anidro_empresa.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/tancagem_anidro_empresa.csv')
processed_blob.upload_from_filename('tancagem_anidro_empresa.csv')

                                          Razão Social  Capacidade de Tancagem
106                       U.S.J. - ACUCAR E ALCOOL S/A                171000.0
80                            PEDRA AGROINDUSTRIAL S/A                166700.0
84                                  RAIZEN ENERGIA S.A                140257.0
111             USINA ALTA MOGIANA S/A-ACUCAR E ALCOOL                115000.0
97                                 SJC BIOENERGIA LTDA                100000.0
..                                                 ...                     ...
115        USINA BOM JESUS S/A EM RECUPERACAO JUDICIAL                  3000.0
133                             USINA SANTA MARIA LTDA                  3000.0
152  ZIHUATANEJO DO BRASIL ACUCAR E ALCOOL S.A EM R...                  2000.0
45   COOPERATIVA DOS PRODUTORES DE CANA DE ACUCAR D...                  1990.0
51                           DESTILARIA NOVA ERA LTDA.                  1316.0

[153 rows x 2 columns]


In [ ]:
df_nulos_removidos_hidratado = df_tancagem.dropna(subset=['Tancagem Etanol Hidratado (M³)'])

tancagem_hidratado_empresa = df_nulos_removidos_hidratado.groupby('Razão Social')['Tancagem Etanol Hidratado (M³)'].max().reset_index(name = "Capacidade de Tancagem")
tancagem_hidratado_empresa = round(tancagem_hidratado_empresa, 2)
tancagem_hidratado_empresa = tancagem_hidratado_empresa.sort_values(by='Capacidade de Tancagem', ascending = False)
print(tancagem_hidratado_empresa)
tancagem_anidro_empresa.to_csv('tancagem_hidratado_empresa.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/tancagem_hidratado_empresa.csv')
processed_blob.upload_from_filename('tancagem_hidratado_empresa.csv')

                                          Razão Social  Capacidade de Tancagem
201                                   SAO MARTINHO S/A                259000.0
56                          CERRADINHO BIOENERGIA S.A.                220000.0
181                                 RAIZEN ENERGIA S.A                141935.0
180             RAIZEN CENTROESTE ACUCAR E ALCOOL LTDA                141935.0
96                             DELTA SUCROENERGIA S.A.                136000.0
..                                                 ...                     ...
158                              MINAS BIOENERGIA LTDA                    30.0
164  NEXUS PRODUCAO E COMERCIALIZACAO DE BIOENERGIA...                    15.0
155                  LOUIS DREYFUS COMPANY BRASIL S.A.                    15.0
213           TRIO COMERCIO DE ACUCAR E LOGISTICA LTDA                    15.0
58         CHS AGRONEGOCIO - INDUSTRIA E COMERCIO LTDA                     5.0

[293 rows x 2 columns]


**12** - Média de tancagem por região

In [ ]:
tancagem_anidro_media = df_nulos_removidos.groupby('Região')['Tancagem Etanol Anidro (M³)'].mean().reset_index(name = "Capacidade de Tancagem")
tancagem_anidro_media = round(tancagem_anidro_media, 2)
tancagem_anidro_media = tancagem_anidro_media.sort_values(by='Capacidade de Tancagem', ascending = False)
print(tancagem_anidro_media)
tancagem_anidro_empresa.to_csv('tancagem_anidro_media.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/tancagem_anidro_media.csv')
processed_blob.upload_from_filename('tancagem_anidro_media.csv')

         Região  Capacidade de Tancagem
3       SUDESTE                40497.12
2         NORTE                37700.00
0  CENTRO OESTE                27391.60
4           SUL                26066.63
1      NORDESTE                11432.52


In [ ]:
tancagem_hidratado_media = df_nulos_removidos_hidratado.groupby('Região')['Tancagem Etanol Hidratado (M³)'].max().reset_index(name = "Capacidade de Tancagem")
tancagem_hidratado_media = round(tancagem_hidratado_media, 2)
tancagem_hidratado_media = tancagem_hidratado_media.sort_values(by='Capacidade de Tancagem', ascending = False)
print(tancagem_hidratado_media)
tancagem_anidro_empresa.to_csv('tancagem_hidratado_media.csv', index=False)
bucket.blob('Dados_tratados/').upload_from_string('')

processed_blob = bucket.blob('Dados_tratados/tancagem_hidratado_media.csv')
processed_blob.upload_from_filename('tancagem_hidratado_media.csv')

         Região  Capacidade de Tancagem
0  CENTRO OESTE                259000.0
3       SUDESTE                226670.0
4           SUL                 90700.0
1      NORDESTE                 49105.0
2         NORTE                 20400.0
